<a href="https://colab.research.google.com/github/juanfranbrv/curso-langchain/blob/main/3.%20Analizadores%20de%20salida%20(Output%20parsers).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. Output parsers en Langchain**
---
Imagina que le preguntas a un LLM "¿Cuáles son los tres planetas más cercanos al sol?" y te responde: "Mercurio, Venus y la Tierra son los planetas más cercanos al sol". Si bien la respuesta es correcta para un humano, para que tu programa pueda usar esa información, lo ideal sería tenerla en un formato más manejable, como una lista o un objeto JSON. Aquí es donde entran en juego los Output Parsers.  

Los Output Parsers te permiten “forzar” o “guiar” al modelo para que devuelva la información según un formato deseado (por ejemplo, un JSON con campos específicos, una lista, etc).  

Convertir el texto libre, en información organizada, en algo que puedes usar directamente en tu código.



```
# Sin output parser: texto plano
"Tom Hanks ha actuado en Forrest Gump y Saving Private Ryan"

# Con output parser: estructura definida
{
  "actor": "Tom Hanks",
  "peliculas": ["Forrest Gump", "Saving Private Ryan"]
}
```

La estructuras de datos de salida son entre otras:

- Objetos JSON
- Modelos Pydantic
- Listas
- Diccionarios
- Enumeraciones


LangChain ofrece una variedad de Output Parsers preconstruidos para diferentes necesidades. Algunos de los mas usados son estos:

- **StrOutputParser:** Convierte la salida a string

- **CommaSeparatedListOutputParser**: Convierte la salida en una lista separada por comas. Útil para generar listas de elementos

- **EnumOutputParser**: Restringe la salida a un conjunto predefinido de valores. Perfecto para categorías o estados limitados. Idela cuedo se desea que el LLM elija de un conjunto de opciones.

- **JsonOutputParser** : Transforma la salida directamente en formato JSON. Ideal para respuestas estructuradas simples. Dos variantes principales:

    - SimpleJsonOutputParser
    - JsonOutputParser

- **DatetimeOutputParser**: Extrae y formatea fechas y horas. Útil para parsear información temporal

- **StructuredOutputParser**: Permite definir esquemas de salida más complejos. Configurable con múltiples campos

- **PydanticOutputParser**: Convierte la salida en objetos Pydantic. Permite definir estructuras de datos complejas. Gran flexibilidad para validación

- **OutputFixingParser**: Intenta corregir salidas mal formateadas. Útil cuando el modelo no genera la estructura perfecta.


Puedes ver todos los OutputParsers disponibles aquí:
https://python.langchain.com/docs/concepts/output_parsers/


# `with_structured_output`

Es un método nativo que aprovecha capacidades del modelo. Utiliza capacidades de llamada de función (function calling) del modelo. Es más eficiente y preciso. Esta soportado por modelos avanzados como OpenAI, Anthropic, Groq.

Lo trataremos al final de este cuaderno.

☝🏻 La mayor parte de modelos soportan esta función y es el futuro de la extracción estructurada en LangChain. Priorízalo cuando puedas.

Puede consultarse una lista de modelos y sus capacidades aquí:
https://python.langchain.com/docs/integrations/chat/



Crear un ejemplo que dada una receta la presente estructurada en ingrdientes, pasos, etc
Esta en este video https://www.youtube.com/watch?v=lbWxastyWPw






#**0. Preparando el entorno del cuaderno**
----
Configuramos el entorno de trabajo para utilizar LangChain con distintos modelos de lenguaje (LLMs).

- Obtenemos las claves API para acceder a los servicios de OpenAI, Groq, Google y Hugging Face.

- Instalamos la librería LangChain y las integraciones necesarias para cada uno de estos proveedores.

- Importamos las clases específicas de LangChain que permiten crear plantillas de prompts e interactuar con los diferentes modelos de lenguaje, dejándolo todo listo para empezar a desarrollar aplicaciones basadas en LLMs. (Este codigo se explico con detalle en el primer cuaderno)

Comenta (#) las librerias y modelos que no desees usar.


In [ ]:
%%capture --no-stderr

# Importar la librería `userdata` de Google Colab.
# Esta librería se utiliza para acceder a datos de usuario almacenados de forma segura en el entorno de Colab.
from google.colab import userdata

# Obtener las claves API de diferentes servicios desde el almacenamiento seguro de Colab.
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')
GROQ_API_KEY=userdata.get('GROQ_API_KEY')
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
HUGGINGFACEHUB_API_TOKEN=userdata.get('HUGGINGFACEHUB_API_TOKEN')

# Instalar las librerías necesarias usando pip.
# El flag `-qU` instala en modo silencioso (`-q`) y actualiza las librerías si ya están instaladas (`-U`).
%pip install langchain -qU  # Instalar la librería principal de LangChain.

# Instalar las integraciones de LangChain con diferentes proveedores de LLMs.
%pip install langchain-openai -qU
%pip install langchain-groq -qU
%pip install langchain-google-genai -qU
%pip install langchain-huggingface -qU

# Instalamos Rich para mejorar la salida
%pip install rich -qU

# Importar las clases necesarias de LangChain para crear plantillas de prompt.
# `ChatPromptTemplate` es la clase base para plantillas de chat.
# `SystemMessagePromptTemplate` se usa para mensajes del sistema (instrucciones iniciales).
# `HumanMessagePromptTemplate` se usa para mensajes del usuario.
from langchain.prompts import PromptTemplate, ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

# Importar las clases para interactuar con los diferentes LLMs a través de LangChain.
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_huggingface import HuggingFaceEndpoint

# Importamos las librerias para formatear mejor la salida
from IPython.display import Markdown, display
from rich import print as rprint

## **StrOutputParser**
---

`StrOutputParser` es el output parser más simple de LangChain. Su función principal es convertir la salida del modelo de lenguaje directamente en una cadena de texto sin realizar ninguna transformación estructural.

Características principales:
- Convierte la salida del modelo a texto plano
- No realiza ninguna validación o estructuración
- Útil cuando solo necesitas el texto sin procesar
- Muy ligero y directo

Casos de uso tipicos: Resumenes, traducciones simples, ...

## Ejemplo: Generador de Resúmenes Ejecutivos

A partir de un ejemplo de ventas (simulado) deseamos obtener un informe sobre el mismo.



In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# Configuramos el modelo
modelo1 = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY)
modelo2 = ChatGroq(model="qwen-qwq-32b", api_key=GROQ_API_KEY)

# Creamos un prompt para generar un resumen ejecutivo
prompt_template = ChatPromptTemplate.from_messages([
    ("system", "Eres un asistente experto en crear resúmenes ejecutivos concisos y claros."),
    ("human", "Genera un resumen ejecutivo sobre el siguiente informe de ventas: {informe}")
])

# Configuramos el output parser de tipo String
output_parser = StrOutputParser()

# Ejemplo de uso
informe_ventas = """
Ventas del Q1 2024:
- Ingresos totales: $1.5M
- Crecimiento interanual: 22%
- Producto más vendido: Software de gestión
- Principales mercados: Tecnología y Finanzas
"""

prompt = prompt_template.format_prompt(informe=informe_ventas)
respuesta = modelo1.invoke(prompt).content
rprint(f"[bold bright_cyan]Respuesta del modelo:\n {respuesta}")
rprint("-----")
respuesta_formateada = output_parser.parse(respuesta)
rprint(f"[bold dark_sea_green4]Resumen generado:\n {respuesta_formateada}")


En cadenas muy simples donde solo esperas texto plano y no necesitas un control estricto sobre el tipo de dato, la diferencia práctica entre usar StrOutputParser explícitamente y no usar ningún OutputParser puede ser mínima. En muchos casos, obtendrás una salida de texto en ambos escenarios.

Sin embargo, usar StrOutputParser explícitamente es una buena práctica ya que mejora la claridad y legibilidad del código.
   
Ahora que entendemos cómo obtener texto plano, veamos cómo podemos empezar a estructurar la salida.

**Pero antes necesitamos conocer y manejar dos conceptos relacionados...**



# `Partial variables`

Las partial_variables son un mecanismo en LangChain para pre-rellenar variables en un prompt de manera parcial, antes de su uso final.  

Piénsalo de esta manera: un PromptTemplate es como una plantilla de texto con "huecos" que necesitas llenar para crear un prompt completo para el LLM. Hay dos formas principales de llenar estos huecos:

-   **input\_variables:** Estas son las variables que **cambian** cada vez que utilizas el prompt. Son los datos específicos que quieres que el LLM procese en cada llamada.
    
-   **partial\_variables:** Estas son las variables que tienen un valor **fijo** o **predefinido** para un uso particular del PromptTemplate. No cambian con cada llamada a la cadena o LLM que usa este prompt.

En este ejemplo nuestro PromptTemplate tiene 4 "huecos" o 4 variables. Pero al crearlo hemos precargado 3 de ellas con valores via partial variables. De esta al invocar el prompt (es un runnable !!) basta que pasemos una (tema).   
Sin imbargo podriamos pasar tambien las restanteas...

In [22]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# Prompt con partial_variables
prompt = PromptTemplate(
    template="Eres un {role} especializado en {area}. {instrucciones}",
    input_variables=["tema"],
    partial_variables={
        "role": "analista",
        "area": "tecnología",
        "instrucciones": "Proporciona un análisis detallado y objetivo."
    }
)

# Uso del prompt
resultado = prompt.invoke({"tema": "Inteligencia Artificial"})
print(resultado)

resultado = prompt.invoke({"tema": "Inteligencia Artificial", "instrucciones": "Contesta con un pareado que rime"})
print(resultado)


text='Eres un analista especializado en tecnología. Proporciona un análisis detallado y objetivo.'
text='Eres un analista especializado en tecnología. Contesta con un pareado que rime'


Un uso tipico de las partial_variables es usarlas para introducir en el prompt las instrucciones de l formato que proporciona Langchain

# `.get_format_instructions()`

**Obtener instrucciones de formato (opcional pero recomendado):** Muchos parsers tienen un método get\_format\_instructions() que devuelve texto que puedes incluir en tu prompt para guiar al LLM sobre el formato esperado.



In [23]:
from langchain.output_parsers import CommaSeparatedListOutputParser

# Crear el ListOutputParser
output_parser = CommaSeparatedListOutputParser()

# Obtener el formato de instrucciones del parser
format_instructions = output_parser.get_format_instructions()
format_instructions

'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

Podriamos redactar nosotros mismos las instrucciones ? SI, sin duda. Esto es solo una funcion de utilidad que nos proporciona Langchain. Teoricamnte disponemos de esta forma de una redaccion tecnicamente correcta.

# **CommaSeparatedListOutputParser:**
---

El `CommaSeparatedListOutputParser` en Langchain es un OutputParser **simple pero muy útil** diseñado para **interpretar la salida de un modelo de lenguaje (LLM) como una lista de elementos separados por comas.** Su función principal es tomar el texto generado por el LLM y **transformarlo en una lista de strings de Python**, donde cada string representa un elemento de la lista original que estaba separado por comas en el texto del LLM.

  

Imagina que le pides a un LLM que te dé "tres colores primarios separados por comas". Podrías esperar una respuesta como:

"rojo, azul, amarillo"

El CommaSeparatedListOutputParser toma esta cadena "rojo, azul, amarillo" y la procesa de la siguiente manera:

- **Divide la cadena:** Utiliza la coma (,) como delimitador para dividir la cadena en partes más pequeñas.
    
- **Elimina espacios en blanco (opcional):** Puede configurarse para eliminar espacios en blanco al principio y al final de cada parte extraída. Por defecto, suele hacerlo para limpiar la lista resultante.
    
- **Crea la lista:** Cada parte resultante se convierte en un elemento de una lista de Python.
    
-   **Casos de uso:** Obtener listas de elementos, como nombres, ideas, pasos a seguir, o categorías.
    
-   **Ventaja:** Simple y efectivo para extraer listas.




## Ejemplo: Lista de ingredientes
Queremos obtener una lista de ingredientes para hacer una pizza casera y solo nos interesa la lista de ingredientes, pues la procesaremos posteriormente de alguan forma.

👀Observa el uso de las partial_variables paara introducir en el prompt las instrucciones de formato

👀Observa tambien el tipo de las dos respuestas. El primero es un string y no podriamos iterarlo. El segundo es una lista de python, si podemos iterarla.

In [48]:
from langchain.output_parsers import CommaSeparatedListOutputParser


# Crear el ListOutputParser
output_parser = CommaSeparatedListOutputParser()

# Crear el prompt
prompt_template = PromptTemplate(
    template="Genera una lista de ingredientes para hacer {receta} casera. Solo lista los ingredientes, uno por línea. Sin opciones\n{format_instructions}\n",
    input_variables=[],
    partial_variables={"format_instructions": output_parser.get_format_instructions()}
)

# Inicializar el modelo de lenguaje
modelo = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY, temperature=0)

# Generar la salida
prompt = prompt_template.format(receta="pizza")
respuesta = modelo.invoke(prompt).content

# Parsear la salida
respuesta_formateada = output_parser.parse(respuesta)

# Mostrar los resultados
rprint(f"Respuesta del modelo SIN FORMATEAR:\n [bold bright_cyan]{respuesta}")
rprint(type(respuesta))
rprint("\n\n")
rprint(f"Respuesta del modelo FORMATEADA:\n [bold spring_green3]{respuesta_formateada}")
rprint(type(respuesta_formateada))


Respuesta del modelo SIN FORMATEAR:
 harina, agua, levadura, sal, aceite de oliva, salsa de tomate, queso mozzarella, orégano, pepperoni, pimientos, 
champiñones, cebolla

<class 'str'>

Respuesta del modelo FORMATEADA:
 ['harina', 'agua', 'levadura', 'sal', 'aceite de oliva', 'salsa de tomate', 'queso mozzarella', 'orégano', 
'pepperoni', 'pimientos', 'champiñones', 'cebolla']

<class 'list'>

## Ejemplo: Lista de etiquetas  
Deseamos que le modelo genere una lista de hastags (o etiquetas) a partir del tema de un articulo

In [57]:
from langchain.output_parsers import CommaSeparatedListOutputParser


# Crear el ListOutputParser
output_parser = CommaSeparatedListOutputParser()

# Crear el prompt
prompt_template = PromptTemplate(
    template="""Dame 5 etiquetas relevantes para un post de blog sobre: {tema}.
                Las etiquetas deben estar separadas por comas.""",
    input_variables=["tema"],
    partial_variables={"format_instructions": output_parser.get_format_instructions()}
)


# Inicializar el modelo de lenguaje
modelo = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY, temperature=0)


# Formatear el prompt con el tema del blog
prompt = prompt_template.format(tema="Recetas de cocina vegana fáciles y rápidas para principiantes")

# Obtener la salida del LLM
respuesta = modelo.invoke(prompt).content

# Parsear la salida
respuesta_formateada = output_parser.parse(respuesta)

# Mostrar los resultados
rprint(f"Respuesta del modelo SIN FORMATEAR:\n [bold bright_cyan]{respuesta}")
rprint(type(respuesta))

rprint("\n\n")

rprint(f"Respuesta del modelo FORMATEADA:\n [bold spring_green3]{respuesta_formateada}")
rprint(type(respuesta_formateada))

rprint("\n\n")

# Imprimir las etiquetas generadas
print("Etiquetas sugeridas:")
for etiqueta in respuesta_formateada:
    rprint(f"[bold spring_green3] - {etiqueta.strip()}")


Respuesta del modelo SIN FORMATEAR:
 recetas veganas, cocina fácil, principiantes, comida saludable, recetas rápidas

<class 'str'>

Respuesta del modelo FORMATEADA:
 ['recetas veganas', 'cocina fácil', 'principiantes', 'comida saludable', 'recetas rápidas']

<class 'list'>

Etiquetas sugeridas:


 - recetas veganas

 - cocina fácil

 - principiantes

 - comida saludable

 - recetas rápidas

# **EnumOutputParser**

`EnumOutputParser` es un tipo de output parser en LangChain que se utiliza para restringir la salida de un modelo a un conjunto predefinido de valores. Esto es útil cuando se desea que la respuesta del modelo pertenezca a un conjunto específico de opciones, como categorías, estados o tipos.

### Características Principales:

-   **Restricción de Valores**: Permite definir un conjunto limitado de opciones que el modelo puede devolver.
-   **Validación Automática**: Si la salida del modelo no coincide con las opciones definidas, se puede manejar como un error.
-   **Facilita la Consistencia**: Asegura que las respuestas sean coherentes y dentro de un rango esperado.



## Ejemplo de Caso de Uso: Clasificación de Sentimientos

Imaginemos que estamos construyendo un sistema que clasifica el sentimiento de comentarios de clientes sobre un producto. Queremos que el modelo devuelva solo tres categorías: "positivo", "negativo" y "neutral".

In [67]:
from enum import Enum
from langchain.output_parsers import EnumOutputParser

# Definimos las opciones de sentimiento
class Sentimientos(Enum):
    POSITIVO = "positivo"
    NEGATIVO = "negativo"
    NEUTRAL = "neutral"

# Crear el EnumOutputParser
output_parser = EnumOutputParser(enum=Sentimientos)

# Crear el prompt
prompt_template = PromptTemplate(
    template="Clasifica el siguiente comentario: {comentario}",
    input_variables=["comentario"],
    partial_variables={"format_instructions": output_parser.get_format_instructions()}
)


# Inicializar el modelo de lenguaje
modelo = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY, temperature=0)


# Ejemplo de uso
comentario_cliente = "Me encanta este producto, es increíble y funciona muy bien."

prompt = prompt_template.format(comentario=comentario_cliente)

# Clasificamos el sentimiento
respuesta = modelo.invoke(prompt).content

# Parsear la salida
respuesta_formateada = output_parser.parse(respuesta)


rprint("Sentimiento Clasificado:")
rprint(respuesta_formateada)


OutputParserException: Response 'El comentario se puede clasificar como **positivo**. Expresa satisfacción y aprecio por el producto, destacando su efectividad.' is not one of the expected values: ['positivo', 'negativo', 'neutral']
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 

## **StructuredOutputParser**
Un analizador de salida estructurado está diseñado para extraer y estructurar información de texto no estructurado o semiestructurado. Esto es adecuado para situaciones que requieren la extracción de múltiples campos.

Permite analizar la salida del LLM en una estructura predefinida. Le proporcionas un esquema (generalmente una lista de ResponseSchema que definen los campos esperados) y el parser intenta extraer la información y mapearla a ese esquema.

Aunque el analizador Pydantic (explicado a continuación) ofrece capacidades más sólidas, StructuredOutputParser es ventajoso para su uso con modelos más simples.

**Casos de uso:** Extraer información específica de un texto, como atributos de un producto, detalles de un evento, o datos de contacto. Es muy versátil para obtener datos estructurados.

**Ventaja:** Flexible y permite definir la estructura esperada de la salida.

Veamos un ejemplo en el que el resultado será un diccionario estructurado con los campos nombre, edad y email, listo para ser utilizado en la aplicación.

In [ ]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

#Creamos un esquema de respuesta (ResponseSchema) para cada campo que queremos extraer:
response_schemas = [
    ResponseSchema(name="nombre", description="El nombre del usuario"),
    ResponseSchema(name="edad", description="La edad del usuario"),
    ResponseSchema(name="email", description="El correo electrónico del usuario")
]

# Crear el StructuredOutputParser
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [ ]:
# Crear el prompt
prompt = PromptTemplate(
    template="Genera información de un usuario ficticio.\n{format_instructions}\n",
    input_variables=[],
    partial_variables={"format_instructions": format_instructions}
)

### Sobre las `partial variables`
Las **partial\_variables** dentro de un PromptTemplate son una forma de **pre-cargar o fijar ciertos valores dentro de la plantilla del prompt antes de que se proporcionen las variables de entrada principales, es decir cuando se crea el propmpt template y antes de que se formatee**

Piénsalo de esta manera: un PromptTemplate es como una plantilla de texto con "huecos" que necesitas llenar para crear un prompt completo para el LLM. Hay dos formas principales de llenar estos huecos:

-   **input\_variables:** Estas son las variables que **cambian** cada vez que utilizas el prompt. Son los datos específicos que quieres que el LLM procese en cada llamada.
    
-   **partial\_variables:** Estas son las variables que tienen un valor **fijo** o **predefinido** para un uso particular del PromptTemplate. No cambian con cada llamada a la cadena o LLM que usa este prompt.
    

**En el codigo anterior:**



```
prompt = PromptTemplate(
            template="Genera información de un usuario ficticio.\n{format_instructions}\n",
            input_variables=[],
            partial_variables={"format_instructions": format_instructions}
```
En otras palabras, son una forma de incluir en el prompt template una parte fija, pero mediante una variable, lo que proporciona mas flexibilidad programatica.
En este caso metemos en el prompt, las instruciones que nos ha devuelto el parser.

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY, temperature=1)

respuesta = llm.invoke(prompt.format()) # la llamada no requiere ningun parámetro
respuesta_analizada = output_parser.parse(respuesta.content)
respuesta_analizada



Vamos a diseñar una consulta para un modelo de lenguaje (LLM) que sirva como base para generar un test de preguntas sobre un tema específico. Proporcionaremos el tema y un nivel de dificultad (bajo, medio, alto), y el LLM deberá generar el texto de la pregunta, tres opciones de respuesta y el índice de la respuesta correcta. La respuesta debe estar estructurada en un diccionario con el siguiente formato:

```
{
    "pregunta": "Texto de la pregunta",
    "opciones": ["Opción 1", "Opción 2", "Opción 3"],
    "respuesta_correcta": índice_de_la_opción_correcta
}
```

Este formato permitirá una fácil interpretación y uso de la pregunta generada.


In [ ]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

#Creamos un esquema de respuesta (ResponseSchema) para cada campo que queremos extraer:
response_schemas = [
            ResponseSchema(name="pregunta", description="Texto de la pregunta generada."),
            ResponseSchema(name="opciones", description="Lista de tres opciones de respuesta."),
            ResponseSchema(name="respuesta_correcta", description="Índice de la opción correcta (0, 1 o 2).")
                    ]

# Crear el StructuredOutputParser
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# Obtener el formato de instrucciones del parser
format_instructions = output_parser.get_format_instructions()
format_instructions

# Crear el prompt
texto_prompt = """
        Genera una pregunta de test sobre el tema: {tema}.
        El nivel de dificultad debe ser: {nivel}.
        La pregunta debe tener tres opciones de respuesta y una respuesta correcta.

        {format_instructions}
        """

prompt_template = PromptTemplate(
    template=texto_prompt,
    input_variables=["tema", "nivel"],
    partial_variables={"format_instructions": format_instructions}
)

# Generar el prompt para el LLM
prompt = prompt_template.format(tema="LangChain", nivel="medio")
print("Prompt generado:\n", prompt)

# Instanciamos el modelo
llm = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY, temperature=1)

# Hacemos la llamada
respuesta = llm.invoke(prompt)

# Analizamos la respuesta
respuesta_analizada = output_parser.parse(respuesta.content)
respuesta_analizada

## **PydanticOutputParser**

Convierte la salida a un modelo de datos Pydantic.

Si buscas simplicidad y rapidez, StructuredOutputParser es una buena opción pero si necesitas validación de datos robusta y estás utilizando Pydantic en tu proyecto, PydanticOutputParser es la mejor alternativa.

| Característica              | StructuredOutputParser               | PydanticOutputParser               |
|-----------------------------|--------------------------------------|-------------------------------------|
| **Definición de esquema**    | Lista de `ResponseSchema`            | Clase Pydantic (`BaseModel`)        |
| **Validación de tipos**      | No estricta                         | Estricta (usando Pydantic)          |
| **Uso**                     | Sencillo y rápido                   | Más verboso pero organizado         |
| **Integración con Pydantic**| No                                  | Sí                                  |
| **Recomendado para**         | Prototipos rápidos                  | Aplicaciones robustas y complejas   |

Veamos como realizar el ejemplo anterior con PydanticOutputParser

In [ ]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List

# Definimos el modelo Pydantic para la respuesta
class PreguntaTest(BaseModel):
    pregunta: str = Field(description="Texto de la pregunta generada.")
    opciones: List[str] = Field(description="Lista de tres opciones de respuesta.")
    respuesta_correcta: int = Field(description="Índice de la opción correcta (0, 1 o 2).")

# Crear el PydanticOutputParser
output_parser = PydanticOutputParser(pydantic_object=PreguntaTest)

# Obtener el formato de instrucciones del parser
format_instructions = output_parser.get_format_instructions()
format_instructions

# Crear el prompt
texto_prompt = """
Genera una pregunta de test sobre el tema: {tema}.
El nivel de dificultad debe ser: {nivel}.
La pregunta debe tener tres opciones de respuesta y una respuesta correcta.

{format_instructions}
"""

prompt_template = PromptTemplate(
    template=texto_prompt,
    input_variables=["tema", "nivel"],
    partial_variables={"format_instructions": format_instructions}
)

# Generar el prompt para el LLM
prompt = prompt_template.format(tema="LangChain", nivel="medio")
print("Prompt generado:\n", prompt)

# Instanciamos el modelo
llm = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY, temperature=1)

# Hacemos la llamada
respuesta = llm.invoke(prompt)

# Analizamos la respuesta
respuesta_analizada = output_parser.parse(respuesta.content)
print("Respuesta analizada:\n", respuesta_analizada)


Veamos otro ejemplo con Pydantic. Deseamos obtener la bibliografia ordenada del autor indicado. deseamos una lista ordenada con el año y el titulo de cada libro.





In [ ]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List

# Definimos el modelo Pydantic para la bibliografía
class Libro(BaseModel):
    año: int = Field(description="Año de publicación del libro.")
    título: str = Field(description="Título del libro.")

class BibliografiaAutor(BaseModel):
    libros: List[Libro] = Field(description="Lista de libros publicados por el autor.")

# Crear el PydanticOutputParser
output_parser = PydanticOutputParser(pydantic_object=BibliografiaAutor)

# Obtener el formato de instrucciones del parser
format_instructions = output_parser.get_format_instructions()
format_instructions

# Crear el prompt
texto_prompt = """
Proporciona la bibliografía del autor: {autor}.
La respuesta debe incluir solo el año y el título de cada libro.
No incluyas información adicional.

{format_instructions}
"""

prompt_template = PromptTemplate(
    template=texto_prompt,
    input_variables=["autor"],
    partial_variables={"format_instructions": format_instructions}
)

# Generar el prompt para el LLM
autor="George R. R. Martin"
prompt = prompt_template.format(autor=autor)
print("Prompt generado:\n", prompt)

# Instanciamos el modelo
llm = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY, temperature=1)

# Hacemos la llamada
respuesta = llm.invoke(prompt)

display(Markdown(f"**Respuesta del LLM sin parsear:**\n{respuesta.content}"))

# Analizamos la respuesta
respuesta_analizada = output_parser.parse(respuesta.content)

# Imprimir la bibliografía de forma legible
# La función enumerate toma un iterable (como una lista) y devuelve un objeto que genera pares de valores
# start=1 , para que el 0 se considere 1
print(f"Bibliografía de {autor}:\n")
for i, libro in enumerate(respuesta_analizada.libros, start=1):
    print(f"Libro {i}:")
    print(f"  Año: {libro.año}")
    print(f"  Título: {libro.título}\n")

print(type(respuesta_analizada))

Observa que la clase del objeto devuelto NO es un diccionario, ni un JSON sino un objeto heredado de la clase que hemos definido con Pydantic !!


## **JsonOutputParser**

JsonOutputParser de LangChain es una herramienta que te permite parsear la salida de un modelo de lenguaje (LLM) en formato JSON. Esto es especialmente útil cuando quieres que el LLM genere una respuesta estructurada en JSON y luego convertir esa respuesta en un objeto de Python (como un diccionario o una lista) para su manipulación.

In [ ]:
from langchain_core.output_parsers import JsonOutputParser


# Definimos el esquema de salida esperado en JSON
schema = {
    "type": "object",
    "properties": {
        "libros": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "año": {"type": "integer", "description": "Año de publicación del libro."},
                    "título": {"type": "string", "description": "Título del libro."}
                },
                "required": ["año", "título"]
            }
        }
    },
    "required": ["libros"]
}

# Crear el JsonOutputParser
output_parser = JsonOutputParser(pydantic_schema=schema)

# Obtener el formato de instrucciones del parser
format_instructions = output_parser.get_format_instructions()

# Crear el prompt
texto_prompt = """
Proporciona la bibliografía del autor: {autor}.
La respuesta debe incluir solo el año y el título de cada libro.
No incluyas información adicional.

{format_instructions}
"""

prompt_template = PromptTemplate(
    template=texto_prompt,
    input_variables=["autor"],
    partial_variables={"format_instructions": format_instructions}
)

# Generar el prompt para el LLM
autor = "Gabriel García Márquez"
prompt = prompt_template.format(autor=autor)
print("Prompt generado:\n", prompt)

# Instanciamos el modelo
llm = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY, temperature=1)

# Hacemos la llamada
respuesta = llm.invoke(prompt)

# Analizamos la respuesta
respuesta_analizada = output_parser.parse(respuesta.content)
print("Respuesta analizada:\n", respuesta_analizada)
print(type(respuesta_analizada))

Observa que el tipo de la respuesta obtenida es direcatmente un diccionario de Python

## **RegexOutputParser**

In [ ]:
from enum import Enum
from langchain.output_parsers import EnumOutputParser

# Definir las opciones del enumerado
class ColoresOjos(Enum):
    MARRÓN = "marrón"
    AVELANA = "avellana"
    ÁMBAR = "ámbar"
    VERDE = "verde"
    AZUL = "azul"
    GRIS = "gris"


# Crear el EnumOutputParser
output_parser = EnumOutputParser(enum=ColoresOjos)

# Crear el prompt
prompt = PromptTemplate(
    template="De que color tenia o tiene los ojos esta persona: {persona}.",
    input_variables=["persona"],
    partial_variables={"format_instructions": output_parser.get_format_instructions()}
)

# Inicializar el modelo de lenguaje
llm = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY, temperature=0.7)

# Generar la salida
input_prompt = prompt.format(persona= "Frank Sinatra")
output = llm.invoke(input_prompt).content.lower()

parsed_output = output_parser.parse(output)

# Parsear la salida
print(f"El género elegido es: {parsed_output}")


En este ejemplo realizamos dos llamadas al modelo. En la primera le pedimos que escoja entre una opcion del Enum, y con ella realizamos la segunda invocación.

In [ ]:
from langchain.output_parsers import EnumOutputParser

from enum import Enum

# Definir los géneros literarios como un Enum
class GenerosLiterarios(Enum):
    FANTASÍA = "fantasía"
    CIENCIA_FICCIÓN = "ciencia ficción"
    MISTERIO = "misterio"
    ROMANCE = "romance"
    HISTÓRICA = "novela histórica"
    NO_FICCIÓN = "no ficción"

# Crear el EnumOutputParser
output_parser = EnumOutputParser(enum=GenerosLiterarios)

# Crear el prompt
prompt = PromptTemplate(
    template="Elige un género literario de la siguiente lista: {generos_literarios}.\nResponde solo con el genero escogido \n{format_instructions}\n",
    input_variables=[],
    partial_variables={
        "generos_literarios": ", ".join([g.value for g in GenerosLiterarios]),
        "format_instructions": output_parser.get_format_instructions()
    }
)

# Inicializar el modelo de lenguaje
llm = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY, temperature=2)

# Generar la salida
input_prompt = prompt.format()
print(input_prompt)
respuesta = llm.invoke(input_prompt).content.lower()
print (respuesta)



In [ ]:
# Parsear la salida
try:
    genero_elegido = output_parser.parse(respuesta)
    print(f"El género elegido es: {genero_elegido.value}")

    # Ahora pedimos una recomendación de libro para el género elegido
    recomendacion_prompt = PromptTemplate(
        template="Recomienda un libro de género {genero}.",
        input_variables=["genero"]
    )
    recomendacion_input = recomendacion_prompt.format(genero=genero_elegido.value)
    recomendacion = llm.invoke(recomendacion_input)

    print(f"Recomendación: {recomendacion.content}")
except ValueError as e:
    print(f"Error: {e}")

No esta demasiado clara la conveniencia de ests OutputParser. Porque no hacer simplemente una eleccion aleatoria  ¿?¿?¿?


```
genero_elegido=random.choice(list(GenerosLiterarios))
```
En lugar de invocar al modelo ¿?¿?¿?





## **BooleanOutputParser**



In [ ]:
from langchain.output_parsers import BooleanOutputParser


# Crear el BooleanOutputParser
output_parser = BooleanOutputParser()

# Crear el prompt
prompt = PromptTemplate(
    template="Responde con 'Yes' o 'No' a la siguiente afirmación: 'El sol es una estrella.'\n{format_instructions}\n",
    input_variables=[],
    partial_variables={"format_instructions":" "}
)

# Inicializar el modelo de lenguaje
llm = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY, temperature=0.7)

# Generar la salida
input_prompt = prompt.format()
respuesta = llm.invoke(input_prompt)

# Parsear la salida
respuesta_parseada = output_parser.parse(respuesta.content)

# Mostrar el resultado
print(respuesta_parseada)

In [ ]:
from langchain.output_parsers import RegexParser


# Definir la expresión regular para extraer nombre y edad
regex_pattern = r"Nombre: (?P<nombre>.+)\nEdad: (?P<edad>\d+)"

# Crear el RegexParser
output_parser = RegexParser(regex=regex_pattern, output_keys=["nombre", "edad"])

# Crear el prompt
prompt = PromptTemplate(
    template="Genera información de una persona ficticia, incluyendo su nombre y edad.\n{format_instructions}\n",
    input_variables=[],
    partial_variables={"format_instructions": ""}
)

# Inicializar el modelo de lenguaje
llm = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY, temperature=0.7)

# Generar la salida
input_prompt = prompt.format()
respuesta = llm.invoke(input_prompt)

# Parsear la salida
respuesta_parseada = output_parser.parse(respuesta.content)

# Mostrar el resultado
print(respuesta_parseada)

## **OutputFixingParser**

## **YamlOutputParser**

## **XmlOutputParser**

## **DateTimeOutputParser**

## **OutputFixingParser**

## **RetrywithError**

# **with_structured_output()**

solo para llm que lo soportan (openai ...)

In [ ]:
from langchain_core.output_parsers.json import SimpleJsonOutputParser
from langchain_openai import ChatOpenAI

prompt = PromptTemplate.from_template(
    'In JSON format, give me a list of {topic} and their '
    'corresponding names in French, Spanish and in a '
    'Cat Language.'
)

model = ChatOpenAI()
chain = prompt | model | SimpleJsonOutputParser()

async for chunk in chain.astream({'topic': 'colors'}):
    print('-')  # noqa: T201
    print(chunk, sep='', flush=True)  # noqa: T201

Referencias:

1. https://freedium.cfd/https://python.plainenglish.io/langchain-in-chains-7-output-parsers-e1a2cdd40cd3

2. https://bobrupakroy.medium.com/harness-llm-output-parsers-for-a-structured-ai-7b456d231834

3. https://cobusgreyling.medium.com/langchain-structured-output-parser-using-openai-c3fe6927beb7

4. https://python.langchain.com/docs/how_to/output_parser_structured/

5. https://www.comet.com/site/blog/mastering-output-parsing-in-langchain/

6. https://www.gettingstarted.ai/how-to-langchain-output-parsers-convert-text-to-objects/

7. https://www.gettingstarted.ai/how-to-extract-metadata-from-pdf-convert-to-json-langchain/  Este es un buen reto

8. https://www.analyticsvidhya.com/blog/2024/11/output-parsers/
